In [1]:
# Cell 1: Import necessary libraries
import os
import sys
import torch
from torch.utils.data import DataLoader
import time
import random
import numpy as np
import yaml
from box import Box
from pprint import pprint

CONFIG_FILENAME = '/home/liranc6/ecg_forecasting/mrDiff/configs/config.yml'

assert CONFIG_FILENAME.endswith('.yml')

with open(CONFIG_FILENAME, 'r') as file:
    config = yaml.safe_load(file)

# Add the parent directory to the sys.path
ProjectPath = config['project_path']
sys.path.append(ProjectPath)

# Add the directory containing the exp module to the sys.path
exp_module_path = os.path.join(ProjectPath, 'mrDiff')
sys.path.append(exp_module_path)

from liran_project.mrdiff.src.parser import Args
from mrDiff.exp.exp_main import Exp_Main

In [2]:
args = Args(CONFIG_FILENAME)
# Now you can use args as needed
pprint(vars(args))

{'config': Box({'project_path': '/home/liranc6/ecg_forecasting', 'general': {'random_seed': 2023, 'evaluate': False, 'tag': None, 'dataset': 'ETTm1', 'features': 'S', 'training_mode': 'ONE', 'interval': 1000}, 'optimization': {'learning_rate': 0.001, 'batch_size': 2048, 'test_batch_size': 32, 'patience': 10, 'weight_decay': 1e-05, 'lradj': '3', 'pct_start': 0.3}, 'hardware': {'num_workers': 0, 'use_gpu': True, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0'}, 'paths': {'checkpoints': './checkpoints/', 'train_data': '/home/liranc6/ecg_forecasting/mrDiff/datasets/SCINet_timeseries/ETT-data/ETT'}, 'data': {'freq': 'h', 'embed': 'timeF', 'cols': [], 'target': -1, 'inverse': False, 'individual': True, 'use_ar_init': False, 'use_residual': True, 'uncertainty': False, 'norm_method': 'z_score', 'normtype': 0}, 'training': {'iterations': {'itr': 1, 'pretrain_epochs': 2, 'train_epochs': 2, 'sample_times': 1}, 'identifiers': {'id_worst': -1, 'focus_variate': -1}, 'sequence': {'seq_len': 336, 'la

In [3]:
# Load the dataset
if args.config['general']['dataset'] == 'ETTm1':
    args.config['paths']['datasets_dir'] = '/home/liranc6/ecg_forecasting/mrDiff/datasets/SCINet_timeseries/ETT-data/ETT'
    args.config['data']['data'] = 'ETTm1'
    args.config['paths']['root_path'] = args.config['paths']['datasets_dir']
    args.config['paths']['data_path'] = 'ETTm1.csv'
    args.config['data']['num_vars'] = 7

In [4]:
# Initialize metrics lists
mae_, mse_, rmse_, mape_, mspe_, rse_, corr_, nrmse_ = [], [], [], [], [], [], [], []

# Ensure the necessary configurations are available
random_seed = args.config.get('general', {}).get('random_seed', 2023)
tag = args.config.get('general', {}).get('tag', None)
dataset = args.config.get('general', {}).get('dataset', 'default_dataset')
features = args.config.get('general', {}).get('features', 'S')

learning_rate = args.config.get('optimization', {}).get('learning_rate', 0.001)
batch_size = args.config.get('optimization', {}).get('batch_size', 64)

seq_len = args.config.get('training', {}).get('sequence', {}).get('seq_len', 336)
label_len = args.config.get('training', {}).get('sequence', {}).get('label_len', 336)
model = args.config.get('training', {}).get('model', {}).get('model', 'default_model')
pred_len = args.config.get('training', {}).get('sequence', {}).get('pred_len', 96)
iterations = args.config.get('training', {}).get('iterations', {}).get('itr', 1)

inverse = args.config.get('data', {}).get('inverse', False)


In [5]:
def run_experiments(iterations, random_seed, model, dataset, features, seq_len,
                    label_len, pred_len, learning_rate, batch_size, inverse, tag, args):
    mae_ = []
    mse_ = []
    rmse_ = []
    mape_ = []
    mspe_ = []
    rse_ = []
    corr_ = []
    nrmse_ = []

    for ii in range(iterations):
        # setting record of experiments

        # random seed
        fix_seed = ii if iterations > 1 else random_seed

        random.seed(fix_seed)
        torch.manual_seed(fix_seed)
        np.random.seed(fix_seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True  # Can change it to False --> default: False
        torch.backends.cudnn.enabled = True

        setting = f"{model}_{dataset}_ft{features}_sl{seq_len}_ll{label_len}_pl{pred_len}_lr{learning_rate}_bs{batch_size}_inv{inverse}_itr{ii}"
        if tag is not None:
            setting += f"_{tag}"

        exp = Exp_Main(args)

        print(f'>>>>>>>start training : {setting}>>>>>>>>>>>>>>>>>>>>>>>>>>')
        exp.train(setting)

        print(f'>>>>>>>testing : {setting}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
        mae, mse, rmse, mape, mspe, rse, corr, nrmse = exp.test(setting, test=1)

        mae_.append(mae)
        mse_.append(mse)
        rmse_.append(rmse)
        mape_.append(mape)
        mspe_.append(mspe)
        rse_.append(rse)
        corr_.append(corr)
        nrmse_.append(nrmse)

        torch.cuda.empty_cache()

    print('Final mean normed: ')
    print('> mae:{:.4f}, std:{:.4f}'.format(np.mean(mae_), np.std(mae_)))
    print('> mse:{:.4f}, std:{:.4f}'.format(np.mean(mse_), np.std(mse_)))
    print('> rmse:{:.4f}, std:{:.4f}'.format(np.mean(rmse_), np.std(rmse_)))
    print('> mape:{:.4f}, std:{:.4f}'.format(np.mean(mape_), np.std(mape_)))
    print('> rse:{:.4f}, std:{:.4f}'.format(np.mean(rse_), np.std(rse_)))
    print('> corr:{:.4f}, std:{:.4f}'.format(np.mean(corr_), np.std(corr_)))
    print('> nrmse:{:.4f}, std:{:.4f}'.format(np.mean(nrmse_), np.std(nrmse_)))

    return {
        'mae': (np.mean(mae_), np.std(mae_)),
        'mse': (np.mean(mse_), np.std(mse_)),
        'rmse': (np.mean(rmse_), np.std(rmse_)),
        'mape': (np.mean(mape_), np.std(mape_)),
        'rse': (np.mean(rse_), np.std(rse_)),
        'corr': (np.mean(corr_), np.std(corr_)),
        'nrmse': (np.mean(nrmse_), np.std(nrmse_))
    }
    
results = run_experiments(iterations, random_seed, model, dataset, features, seq_len,
                            label_len, pred_len, learning_rate, batch_size, inverse, tag, args)

Use GPU: cuda:0


>>>>>>>start training : DDPM_ETTm1_ftS_sl336_ll336_pl96_lr0.001_bs2048_invFalse_itr0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 34129
val 11425
test 11425


IndexError: index 1 is out of bounds for dimension 1 with size 1